## Plan 
this code snippet operates in 2 phases
### Phase 1  : Learning the patterns of a specific subject
(Assuming that a patient is logged in)
1. Actuators
 1. Asks for actuators data
 1. buildes a knn model and saves it
1. ECG
 1. Asks for ECG data while doing different activities
 1. 

In [219]:
import pandas as pd
import numpy as np

Prepare input files (Sensors dummy readings)

In [220]:
def getSubjectNum(fileName) :
        import re
        n = re.findall(r'\d+', fileName)
        return int(n[0])


In [265]:
class GyroSensor:
    def __init__(self,freq=50,rawdataFileName="mHealth_subject3.log"):
        self.frequency=50
        self.rawdata="mHealth_subject3.log"
        self.subjectNum=getSubjectNum(self.rawdata)
        
        self.generateInputFile()
        self.datapool=pd.DataFrame(np.array([]))
    
    def generateInputFile(self):
        X =  np.array([])
        Y =  np.array([])
        df = pd.read_csv(self.rawdata, header=None, delim_whitespace=True)
        df = df[df[23]!=0] #drop 0 labeled features
        readings_df = pd.DataFrame(df[9].copy())
        targets_df = pd.DataFrame(df[23].copy())
        reading_np = readings_df.values

        rps=self.frequency
        activities=range(1,13)
        # #         segmentation
        for i in range(int(readings_df.shape[0]/rps)):
            if  float(targets_df[i*rps:i*rps+rps].mean()) in activities:
                Y=np.append(Y,targets_df[i*rps:i*rps+rps].mean()) #new
                X=np.append(X,readings_df[i*rps:i*rps+rps].values)

        X=X.reshape(-1,rps)
        OutDF = pd.DataFrame(X)
        OutDF['labels']=Y
        OutDF.reset_index
        OutDF.to_csv("Gyro_subject"+str(self.subjectNum), sep='\t',index=False)

    #returns value , target
    def getReading(self,activity=1):
        if self.datapool.empty == True:
            self.datapool= pd.read_csv("Gyro_subject"+str(self.subjectNum), sep='\t',index_col=False)
        readings = self.datapool.loc[df.labels==activity,]
        randIndex = np.random.randint(0,len(readings))
        return np.array(readings.iloc[randIndex,range(0,self.frequency)]), readings.iloc[randIndex,self.frequency]


In [248]:
from sklearn.neighbors import KNeighborsClassifier
#minValuesPerClass = The minimum readings per class required for good accuracy
class ActuatorsBasedModel:
    def __init__(self,sensor=None,minValuesPerClass=30):
        self.model=None
        pass
    def train(self,X,Y):
        self.model=  KNeighborsClassifier(n_neighbors=1)
        self.model.fit(X,Y)
    def predict(self,X):
        if self.model == None:
            print("Not trained yet")
        else:
            return self.model.predict(X)
    def reportAccuracy(self,sensor):
        t=0
        tot=1000
        for i in range(tot):
            x,y = sensor.getReading(np.random.randint(1,13))
            x = np.array(x).reshape(-1,50)
            if(y==self.predict(x)):
                t+=1
        return t/tot

In [223]:

########################################
######Should be embedded somewhere######
X=[]
Y=[]
G = GyroSensor()
for i in range(30):
    for activity in range(1,13):
        x,y = G.getReading(activity)
        X+=list(x)
        Y+=[y]
X=np.array(X).reshape(-1,50)
########################################

a=ActuatorsBasedModel()
a.train(X,Y)
a.reportAccuracy(G)

0.925

In [225]:
a.reportAccuracy(G)

0.924

# ECG

In [264]:
class ECGSensor:
    def __init__(self,freq=50,rawdataFileName="mHealth_subject3.log"):
        self.frequency=50
        self.rawdata="mHealth_subject3.log"
        self.subjectNum=getSubjectNum(self.rawdata)
        
        self.generateInputFile()
        self.datapool=pd.DataFrame(np.array([]))
    
    def generateInputFile(self):
        X =  np.array([])
        Y =  np.array([])
        df = pd.read_csv(self.rawdata, header=None, delim_whitespace=True)
        df = df[df[23]!=0] #drop 0 labeled features
        readings_df = pd.DataFrame(df[3].copy())
        targets_df = pd.DataFrame(df[23].copy())
        reading_np = readings_df.values

        rps=self.frequency
        activities=range(1,13)
        # #         segmentation
        for i in range(int(readings_df.shape[0]/rps)):
            if  float(targets_df[i*rps:i*rps+rps].mean()) in activities:
                Y=np.append(Y,targets_df[i*rps:i*rps+rps].mean()) #new
                X=np.append(X,readings_df[i*rps:i*rps+rps].values)

        X=X.reshape(-1,rps)
        OutDF = pd.DataFrame(X)
        OutDF['labels']=Y
        OutDF.reset_index
        OutDF.to_csv("ECG_subject"+str(self.subjectNum), sep='\t',index=False)

    #returns value , target
    def getReading(self,activity=1):
        if self.datapool.empty == True:
            self.datapool= pd.read_csv("ECG_subject"+str(self.subjectNum), sep='\t',index_col=False)
        readings = self.datapool.loc[df.labels==activity,]
        randIndex = np.random.randint(0,len(readings))
        return np.array(readings.iloc[randIndex,range(0,self.frequency)]), readings.iloc[randIndex,self.frequency]


In [247]:
from sklearn.neighbors import KNeighborsClassifier
#minValuesPerClass = The minimum readings per class required for good accuracy
class ECGBasedModel:
    def __init__(self,sensor=None,minValuesPerClass=30):
        self.model=None
        pass
    def train(self,X,Y):
        self.model=  KNeighborsClassifier(n_neighbors=1)
        self.model.fit(X,Y)
    def predict(self,X):
        if self.model == None:
            print("Not trained yet")
        else:
            return self.model.predict(X)
    def reportAccuracy(self,sensor):
        t=0
        tot=1000
        for i in range(tot):
            x,y = sensor.getReading(np.random.randint(1,13))
            x = np.array(x).reshape(-1,50)
            if(y==self.predict(x)):
                t+=1
        return t/tot

In [260]:

########################################
######Should be embedded somewhere######
X=[]
Y=[]
G = ECGSensor(freq=50,rawdataFileName="mHealth_subject3.log")
for i in range(30):
    for activity in range(1,13):
        x,y = G.getReading(activity)
        X+=list(x)
        Y+=[y]
X=np.array(X).reshape(-1,50)
########################################

a=ECGBasedModel()
a.train(X,Y)
# b=a.predict(X[0].reshape(1, -1))
a.reportAccuracy(G)
# b

array([1.])

# Trainers

In [253]:
def getTrainData(sensor,minReadingsPerActivity=30):
    X=[]
    Y=[]
    for i in range(minReadingsPerActivity):
        for activity in range(1,13):
            x,y = sensor.getReading(activity)
            X+=list(x)
            Y+=[y]
    X=np.array(X).reshape(-1,50)
    return X,Y

## Threading

In [254]:
import threading
import time
import random

In [275]:
Gyro_sensor = GyroSensor(freq=50,rawdataFileName="mHealth_subject3.log")
ECG_sensor = ECGSensor(freq=50,rawdataFileName="mHealth_subject3.log")
A = ActuatorsBasedModel()
X,y = getTrainData(Gyro_sensor)
A.train(X,y)
E = ECGBasedModel()
X,y = getTrainData(ECG_sensor)
E.train(X,y)
while(True):
    activity=np.random.randint(1,13)
    time.sleep(1)
    GYR_databuffer, _ =Gyro_sensor.getReading(activity)
    ECG_databuffer,__=ECG_sensor.getReading(activity)
    Activity_predited_ECG = E.predict(ECG_databuffer.reshape(1,-1))
    Activity_predited_GYR = A.predict(GYR_databuffer.reshape(1,-1))
    print(Activity_predited_ECG,'<-E-|-A->',Activity_predited_GYR)
    if Activity_predited_ECG == Activity_predited_GYR:
        print("**")

[6.] <-E-|-A-> [1.]
[7.] <-E-|-A-> [2.]
[3.] <-E-|-A-> [3.]
**
[9.] <-E-|-A-> [9.]
**
[6.] <-E-|-A-> [7.]
[11.] <-E-|-A-> [11.]
**
[1.] <-E-|-A-> [2.]
[12.] <-E-|-A-> [12.]
**
[11.] <-E-|-A-> [11.]
**
[7.] <-E-|-A-> [6.]
[5.] <-E-|-A-> [5.]
**
[3.] <-E-|-A-> [7.]
[6.] <-E-|-A-> [3.]
[8.] <-E-|-A-> [8.]
**
[6.] <-E-|-A-> [4.]
[4.] <-E-|-A-> [6.]
[9.] <-E-|-A-> [9.]
**
[2.] <-E-|-A-> [6.]
[7.] <-E-|-A-> [7.]
**
[6.] <-E-|-A-> [6.]
**
[8.] <-E-|-A-> [8.]
**
[6.] <-E-|-A-> [9.]
[3.] <-E-|-A-> [10.]
[12.] <-E-|-A-> [10.]
[2.] <-E-|-A-> [4.]
[1.] <-E-|-A-> [1.]
**
[12.] <-E-|-A-> [12.]
**
[5.] <-E-|-A-> [12.]
[12.] <-E-|-A-> [10.]
[9.] <-E-|-A-> [9.]
**
[2.] <-E-|-A-> [10.]
[2.] <-E-|-A-> [2.]
**
[3.] <-E-|-A-> [4.]
[9.] <-E-|-A-> [9.]
**
[3.] <-E-|-A-> [1.]
[4.] <-E-|-A-> [1.]
[10.] <-E-|-A-> [10.]
**


KeyboardInterrupt: 

In [177]:
t=0
tot=10000
for i in range(tot):
    x,y = G.getReading(np.random.randint(1,13))
    x = np.array(x).reshape(-1,50)
    if(y==knn.predict(x)):
        t+=1
print(t/tot)

0.5436


In [ ]:
def ECG_Reader():
    pass

In [ ]:
def Gyro_Reader():
    pass

In [1]:

def fun1():
      for j in range(10):
            print("1")
            time.sleep(random.randint(1,5)/10)

def fun2():
      for i in range(10):
            print("2")
            time.sleep(random.randint(5,10)/10)

if __name__ == "__main__": 
    # creating thread 
    t1 = threading.Thread(target=fun1) 
    t2 = threading.Thread(target=fun2) 
  
    # starting thread 1 
    t1.start() 
    # starting thread 2 
    t2.start() 
  
    # wait until thread 1 is completely executed 
    t1.join() 
    # wait until thread 2 is completely executed 
    t2.join() 
  
    # both threads completely executed 
    print("Done!") 

1
2
1
1
2
1
1
2
1
1
2
1
1
2
1
2
2
2
2
2
Done!
